In [0]:
%sql
create or replace table dev.demo_db.people_tbl(
  id INT,
  firstName STRING,
  lastName STRING
)USING DELTA;

### 1. Insert Column Matching by Positioning (Matching Names Not Mandatory)

In [0]:
%sql
INSERT INTO dev.demo_db.people_tbl
SELECT id,fname,lname
from json.`/Volumes/dev/demo_db/vol1/people.json`

In [0]:
%sql
select * from dev.demo_db.people_tbl

### 2. INSERT - New Columns not allowed

In [0]:
%sql
INSERT INTO dev.demo_db.people_tbl
SELECT id,fname,lname,dob
from json.`/Volumes/dev/demo_db/vol1/people.json`

### 3. OVERWRITE

In [0]:
%sql
INSERT OVERWRITE dev.demo_db.people_tbl
SELECT id,fname,lname,dob
FROM json.`/Volumes/dev/demo_db/vol1/people.json`

### 4. MERGE - Column Matching by Name

In [0]:
%sql
select id,fname,lname from json.`/Volumes/dev/demo_db/vol1/people.json`

In [0]:
%sql
MERGE INTO dev.demo_db.people_tbl tgt
USING (select id,fname,lname from json.`/Volumes/dev/demo_db/vol1/people.json`) src
on tgt.id = src.id
WHEN NOT MATCHED THEN
    INSERT *

### 5. MERGE - New Columns Silently ignored

In [0]:
%sql
SELECT id, fname as firstName, lname as lastName, dob from json.`/Volumes/dev/demo_db/vol1/people.json`

In [0]:
%sql
MERGE INTO dev.demo_db.people_tbl tgt
USING(SELECT id, fname as firstName, lname as lastName, dob from json.`/Volumes/dev/demo_db/vol1/people.json`)src
on tgt.id = src.id
WHEN NOT MATCHED THEN
  INSERT *

In [0]:
%sql
select * from dev.demo_db.people_tbl

### 6. Dataframe Append - Column Matching By Name

In [0]:
people_schema = "id INT,fname STRING,lname STRING"
people_df = spark.read.format("json").schema(people_schema).load("/Volumes/dev/demo_db/vol1/people_2.json")
people_df.write.format("delta").mode("append").saveAsTable("dev.demo_db.people_tbl")

### 7. Dataframe Append - New Columns not allowed

In [0]:
people_schema = "id INT,firstName STRING,lastName STRING,dob STRING"
people_df = spark.read.format("json").schema(people_schema).load("/Volumes/dev/demo_db/vol1/people_2.json")
people_df.write.format("delta").mode("append").saveAsTable("dev.demo_db.people_tbl")